In [1]:
from leeq.core.elements.built_in.qudit_transmon import TransmonElement
from leeq.setups.built_in.setup_simulation_high_level import HighLevelSimulationSetup
import numpy as np
from leeq.theory.simulation.numpy.rotated_frame_simulator import VirtualTransmon
from leeq import ExperimentManager
from labchronicle import Chronicle
Chronicle().start_log()
manager = ExperimentManager()
manager.clear_setups()

virtual_transmon_1 = VirtualTransmon(
    name="VQubit_1",
    qubit_frequency=5040.4,
    anharmonicity=-198,
    t1=70,
    t2=35,
    readout_frequency=9645.5,
    quiescent_state_distribution=np.asarray(
        [
            0.8,
            0.15,
            0.04,
            0.01]))

virtual_transmon_2 = VirtualTransmon(
    name="VQubit_2",
    qubit_frequency=5040. - 123,
    anharmonicity=-197,
    t1=70,
    t2=35,
    readout_frequency=9645.5 + 100,
    quiescent_state_distribution=np.asarray(
        [
            0.8,
            0.15,
            0.04,
            0.01]))

setup = HighLevelSimulationSetup(
    name='HighLevelSimulationSetup',
    virtual_qubits={2: virtual_transmon_1,
                    4: virtual_transmon_2
                    }
)
setup.set_coupling_strength_by_qubit(
    virtual_transmon_1, virtual_transmon_2, coupling_strength=1.5)

manager.register_setup(setup)

configuration_q1 = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4 - 198,
            'channel': 2,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5,
            'channel': 1,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        },
        '1': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9144.41,
            'channel': 1,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1, 2]
        }
    }
}

configuration_q2 = {
    'lpb_collections': {
        'f01': {
            'type': 'SimpleDriveCollection',
            'freq': 5040. - 123,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        },
        'f12': {
            'type': 'SimpleDriveCollection',
            'freq': 5040.4 - 198 - 123,
            'channel': 4,
            'shape': 'blackman_drag',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 0.025,
            'alpha': 425.1365229849309,
            'trunc': 1.2
        }
    },
    'measurement_primitives': {
        '0': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5 + 100,
            'channel': 3,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1]
        },
        '1': {
            'type': 'SimpleDispersiveMeasurement',
            'freq': 9645.5 + 100,
            'channel': 3,
            'shape': 'square',
            'amp': 0.21323904814245054 / 5 * 4,
            'phase': 0.,
            'width': 1,
            'trunc': 1.2,
            'distinguishable_states': [0, 1, 2]
        }
    }
}

qubit_1 = TransmonElement(
        name='test_qubit_1',
        parameters=configuration_q1
    )

qubit_2 = TransmonElement(
        name='test_qubit_2',
        parameters=configuration_q2
    )

[2024-09-25 01:55:50] [INFO] [labchronicle.chronicle] Log started at log/zijian/2024-09/2024-09-25/01.55.50
[2024-09-25 01:55:50] [INFO] [labchronicle.chronicle] Log started at log/zijian/2024-09/2024-09-25/01.55.50


In [ ]:
import os
from k_agents.execution.agent import AutoRun
from k_agents.translation.env import TranslationAgentEnv
from leeq.utils.ai.translation_agent import init_leeq_translation_agent
from leeq.experiments import experiments as exp
root = os.path.dirname(exp.__file__)
init_leeq_translation_agent(root + "/procedures_test")
env = TranslationAgentEnv()
ExperimentManager().status().set_param("Plot_Result_In_Jupyter", False)
duts = [qubit_1, qubit_2]
frequency1 = duts[0].get_c1('f01').get_parameters()["freq"]

#AutoRun("""
#- Run a Two-qubit calibration on `duts`
#""", frequency=frequency1, duts=duts)
AutoRun(
    "Iterative Two-qubit Amplitude test at `frequency` on `duts`",
    frequency=frequency1, duts=duts
)

Adding experiment to memory:   0%|          | 0/15 [00:00<?, ?it/s]

generate_idea_from_procedure:   0%|          | 0/2 [00:00<?, ?it/s]

/Users/zijian/PycharmProjects/LeeQ/venv/lib/python3.10/site-packages/pydantic/main.py:390: UserWarning:

Pydantic serializer warnings:
  Expected `CompletionTokensDetails` but got `dict` with value `{'reasoning_tokens': 0}` - serialized value may not be as expected



_get_stage_from_agent_response:   0%|          | 0/2 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

...

No code suggestion found. Recall more ideas.


Recalling:   0%|          | 0/7 [00:00<?, ?it/s]

   experiment_instance = AutoExperiment_2(instruction="""Iterative Two-qubit Amplitude test at frequency on duts""" , frequency=frequency, duts=duts)  
                          ^^^^
KeyboardInterrupt: 
/Users/zijian/PycharmProjects/KnowleAgents/k_agents/variable_table.py:142: UserWarning:

Error raised when interpreting: experiment_instance = AutoExperiment_2(instruction="""Iterative Two-qubit Amplitude test at frequency on duts""" , frequency=frequency, duts=duts)   
 Error detail: ('KeyboardInterrupt', '   experiment_instance = AutoExperiment_2(instruction="""Iterative Two-qubit Amplitude test at frequency on duts""" , frequency=frequency, duts=duts)  \n                          ^^^^\nKeyboardInterrupt: ') 

('KeyboardInterrupt', '   experiment_instance = AutoExperiment_2(instruction="""Iterative Two-qubit Amplitude test at frequency on duts""" , frequency=frequency, duts=duts)  \nKeyboardInterrupt: ') 


/Users/zijian/PycharmProjects/KnowleAgents/k_agents/execution/agent.py:112: Us

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from leeq.experiments.builtin import ConditionalStarkShiftContinuous

exp = ConditionalStarkShiftContinuous(duts, amp_control=1.2, frequency=7000, amp_target=1.6)
res = exp.get_ai_inspection_results()
res